In [ ]:
import json
import os
import sys
import tarfile
import asyncio

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path

import tengu

### 0) Setup

In [ ]:
# Define our project information
DESCRIPTION = "tengu-py demo notebook"
TAGS = ["qdx", "tengu-py-v1", "demo", "hermes"]
WORK_DIR = Path.home() / "qdx" / "tengu-py-hermes"
OUT_DIR = WORK_DIR / "runs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Set our inputs
PROTEIN_PDB_PATH = WORK_DIR / "test_P.pdb"
LIGAND_SDF_PATH = WORK_DIR / "test_L.sdf"

In [ ]:
# Get our client, for calling modules and using the tengu API
client = await tengu.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
(converted_protein,) = await client.convert("PDB", PROTEIN_PDB_PATH)

print(f"{datetime.now().time()} | Running protein conversion!")

checking <tengu.typedef.EnumKind object> PDB
checking <tengu.typedef.ObjectKind object> /home/machineer/qdx/tengu-py-hermes/test_P.pdb
14:28:03.546047 | Running protein conversion!


In [ ]:
(picked_protein,) = await client.pick_conformer(converted_protein, 0)

checking <tengu.typedef.ObjectKind object> Arg(id=644bcbf4-f5b2-4663-8be3-46fd695b64e1, value=None)


In [ ]:
pp = await picked_protein.get()

In [ ]:
pp_download = await picked_protein.download(filepath="./pp.json")

In [ ]:
help(client.fragment)

Help on function fragment in module tengu.provider:

async fragment(*args: [<class 'pathlib.Path'>, <class 'int'>, typing.Optional[~T]], target: tengu.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX: 'NIX'>, resources: tengu.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=0, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=0, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>]
    Fragments a protein QDX file by cutting the backbone at the alpha-beta carbon bonds, skipping each n steps to increase fragment size
    
    Module version: github:talo/tengu-prelude/e21ba62b15154b9632d29238e9677749febfd088#fragment
    
    QDX Type Description:
    
        input: @Conformer;
    
        steps: u8;
    
        tolerance: f32? 
    
    ->
    
        output: @C

In [ ]:
fragmented_protein = await client.fragment(Path("./pp.json"), 1, None)

loading object


In [ ]:
hermes_energy = await client.hermes_energy(
    fragmented_protein[0],
    None,
    None,
    #target="NIX_SSH_3",
)

In [ ]:
await hermes_energy[0].get()

Exception: Argument was rejected

In [ ]:
i = await hermes_energy[0].info()

AttributeError: 'ArgumentArgument' object has no attribute 'status'

In [ ]:
async for log in client.logs(i.source, "stderr"):
    print(log)

In [ ]:
i.source


UUID('45e9f63c-f1db-47ba-a92a-b9d368eef1c7')